In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
pip install langchain-openai

In [ ]:
import os
import re
import email
import imaplib
import logging
import smtplib
import uuid
from email.header import decode_header
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from docx import Document
import PyPDF2
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
import json

# Configure logging
logging.basicConfig(
    filename='resume_processing.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Configure OpenAI API Key
os.environ["OPENAI_API_KEY"] = "api key"  # Replace with your valid key

class ResumeAgents:
    def __init__(self, llm):
        self.resume_analyzer = Agent(
            role='Senior Resume Analyst',
            goal='Extract key details from resumes and identify critical patterns',
            backstory="""An expert in resume analysis with 10+ years of HR tech experience""",
            llm=llm,
            verbose=True
        )

        self.scoring_agent = Agent(
            role='Resume Scoring Expert',
            goal='Calculate comprehensive scores based on multiple factors',
            backstory="""Data-driven specialist with expertise in technical hiring metrics""",
            llm=llm,
            verbose=True
        )

        self.feedback_agent = Agent(
            role='Career Advisor',
            goal='Generate constructive feedback for candidates',
            backstory="""Professional career coach specializing in tech candidates""",
            llm=llm,
            verbose=True
        )

class ResumeProcessor:
    def __init__(self, llm):
        self.agents = ResumeAgents(llm)

    def extract_text(self, filepath):
        text = ""
        try:
            if filepath.endswith('.pdf'):
                with open(filepath, 'rb') as f:
                    reader = PyPDF2.PdfReader(f)
                    if reader.is_encrypted:
                        reader.decrypt('')
                    text = '\n'.join(page.extract_text() for page in reader.pages)
            elif filepath.endswith('.docx'):
                doc = Document(filepath)
                text = '\n'.join(p.text for p in doc.paragraphs)
        except Exception as e:
            logging.error(f"Error extracting text: {str(e)}")
        return text

    def crewai_analysis(self, text):
        analysis_task = Task(
            description=f"Analyze resume content:\n{text[:5000]}",
            agent=self.agents.resume_analyzer,
            expected_output="JSON with experience, education, skills"
        )

        scoring_task = Task(
            description="Calculate score (0-100) with category breakdown",
            agent=self.agents.scoring_agent,
            expected_output="Numerical score and category percentages"
        )

        feedback_task = Task(
            description="Generate 3 strengths and 3 improvement areas",
            agent=self.agents.feedback_agent,
            expected_output="Formatted feedback paragraph"
        )

        crew = Crew(
            agents=[self.agents.resume_analyzer, self.agents.scoring_agent, self.agents.feedback_agent],
            tasks=[analysis_task, scoring_task, feedback_task],
            verbose=True
        )
        return crew.kickoff()

class EmailHandler:
    @staticmethod
    def send_feedback(sender, password, receiver, score, feedback):
        msg = MIMEMultipart('alternative')
        msg['Subject'] = "Resume Analysis Report"
        msg['From'] = sender
        msg['To'] = receiver
        
        html = f"""
        <html>
        <body>
            <h2>Resume Evaluation</h2>
            <p>Score: {score}/100</p>
            <h3>Feedback:</h3>
            <p>{feedback}</p>
        </body>
        </html>
        """
        
        msg.attach(MIMEText(html, 'html'))
        
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender, password)
            server.sendmail(sender, receiver, msg.as_string())

def process_resumes(email_user, email_pass, sender_email, sender_pass):
    # Create LLM and processor
    llm = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        temperature=0.7
    )
    processor = ResumeProcessor(llm)
    email_handler = EmailHandler()

    # Email setup
    mail = imaplib.IMAP4_SSL('imap.gmail.com')
    mail.login(email_user, email_pass)
    mail.select('INBOX')
    
    _, messages = mail.search(None, 'ALL')
    
    if not os.path.exists("resumes"):
        os.makedirs("resumes")

    for email_id in messages[0].split():
        try:
            _, msg_data = mail.fetch(email_id, '(RFC822)')
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    msg = email.message_from_bytes(response_part[1])
                    for part in msg.walk():
                        if part.get_content_disposition() == 'attachment':
                            filename = part.get_filename()
                            if filename and filename.lower().endswith(('.pdf', '.docx')):
                                # Secure filename handling
                                clean_name = re.sub(r'[^\w\-_. ]', '_', filename)
                                unique_name = f"{uuid.uuid4().hex[:6]}_{clean_name}"
                                filepath = os.path.join("resumes", unique_name)
                                
                                # Save attachment
                                with open(filepath, 'wb') as f:
                                    f.write(part.get_payload(decode=True))
                                
                                # Process resume
                                text = processor.extract_text(filepath)
                                if text:
                                    result = processor.crewai_analysis(text)
                                    print(json.dumps(result, indent=2))
                                    
                                    # Extract candidate email
                                    candidate_email = re.search(r'\b[\w.-]+@[\w.-]+\.\w+\b', text)
                                    if candidate_email:
                                        email_handler.send_feedback(
                                            sender_email,
                                            sender_pass,
                                            candidate_email.group(0),
                                            result.get('score', 0),
                                            result.get('feedback', 'No feedback generated')
                                        )

        except Exception as e:
            logging.error(f"Error processing email {email_id}: {str(e)}")

if __name__ == "__main__":
    process_resumes(
        email_user="user_email",
        email_pass="app_password",
        sender_email="sender_email",
        sender_pass="app_password"
    )
